토픽 모델링은 텍스트 마이닝 기법 중에서 가장 많이 활용되는 기법 중 하나로, 다양한 문서 집합에 내재한 토픽, 즉 주제를 파악할 때 쓰는 방법이다.  

# 1. 토픽 모델링과 LDA의 이해

## 1.1 토픽 모델링이란?

* 토픽 모델링은 '내재된 주제의 분석'을 가능하게 하는 기법.
* 더불어 이러한 주제들이 시간에 따라 어떻게 변화했는지를 살펴보는 '토픽 트렌드'분석을 할 수도 있다.

## 1.2 LDA 모형의 구조

* LDA는 토픽 모델링에서 가장 널리 쓰이는 기본적인 알고리즘이다.  
* LDA의 기본 가정은, 문서들이 쓰여질 떄 그 문서를 구성하는 몇 개의 토픽이 존재하며 각 토픽은 단어의 집합으로 구성됐다는 것이다.
* LDA를 이용한 토픽 모델링의 내용을 요약하자면, 각 문서에 사용된 단어들의 빈도를 측정하고, 이로부터 역으로 모든 문서의 토픽분포와 각 토픽의 단어분포를 추정하는 것이라고 할 수 있다.

## 1.3 모형의 평가와 적절한 토픽 수의 결정

* **Perplexity**는 혼잡도로서 보통 특정한 확률 모형이 실제로 관측되는 값을 얼마나 유사하게 예측해내는지를 평가할 때 사용.
* 값이 작을수록 토픽 모델이 문서집합을 잘 반영한다고 생각하면 된다.
* **토픽 응집도**는 각 토픽에서 상위 비중을 차지하는 단어들이 의미적으로 유사한지를 나타내는 척도이다. 만일 토픽이 단일 주제를 잘 표현한다면 의미적으로 유사한 단어들의 비중이 높을 것이라는 가정에 따라 성능을 표현한다. 이값은 클수록 좋다.

# 2. 사이킷런을 이용한 토픽 모델링

## 2.1 데이터 준비

* 이미 우리가 사용한 바 있는 20뉴스그룹 데이터를 사용해 토픽 모델링을 실습하고자 한다.
* 예측을 할 필요가 없으므로 학습데이터만 사용한다.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space', 'comp.sys.ibm.pc.hardware', 'sci.crypt']

# 학습 데이터셋을 가져옴
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

print('#Train set size: ', len(newsgroups_train.data))
print('#Selected categories: ', newsgroups_train.target_names)

#Train set size:  3219
#Selected categories:  ['alt.atheism', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.crypt', 'sci.space', 'talk.religion.misc']


데이터가 준비되면 카운트 벡터를 생성한다. 사이킷런의 LDA라이브러리는 카운트 벡터를 입력으로 사용한다.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(token_pattern="[\w']{3,}", stop_words='english',
                    max_features=2000, min_df=5, max_df=0.5)

review_cv = cv.fit_transform(newsgroups_train.data)

## 2.2 LDA 토픽 모델링 실행

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
np.set_printoptions(precision=3)

lda = LatentDirichletAllocation(n_components=10,
                               max_iter=5,
                               topic_word_prior=0.1, doc_topic_prior=1.0,
                               learning_method='online',
                               n_jobs=-1,
                               random_state=0)

review_topics = lda.fit_transform(review_cv)
print('#shape of review_topics: ', review_topics.shape)
print('#Sample of review_topics ', review_topics[0])

gross_topic_weights = np.mean(review_topics, axis=0)
print('#Sum of topic weights of documents: ', gross_topic_weights)
print('#shape of topic word distribution: ', lda.components_.shape)

#shape of review_topics:  (3219, 10)
#Sample of review_topics  [0.902 0.007 0.029 0.008 0.007 0.008 0.007 0.007 0.007 0.018]
#Sum of topic weights of documents:  [0.088 0.083 0.084 0.115 0.116 0.126 0.097 0.072 0.07  0.149]
#shape of topic word distribution:  (10, 2000)


In [6]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d: " % topic_idx, end='')
        print(
            ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
        # 위 slicing에서 맨 뒤 -1은 역순을 의미, 역순으로 했을 때 처음부터 n_top_words까지
        
    print()
    
print_top_words(lda, cv.get_feature_names_out(), 10)

Topic #0: com, morality, keith, article, sgi, think, sandvik, caltech, objective, moral
Topic #1: image, file, graphics, files, ftp, available, software, use, data, pub
Topic #2: space, nasa, access, launch, earth, orbit, shuttle, digex, lunar, satellite
Topic #3: article, com, just, like, don't, i'm, university, nntp, host, posting
Topic #4: key, clipper, chip, encryption, com, government, keys, law, use, escrow
Topic #5: scsi, com, bit, ibm, bus, know, windows, thanks, university, card
Topic #6: host, gov, nntp, posting, university, distribution, nasa, ___, world, article
Topic #7: drive, com, disk, hard, controller, drives, dos, tape, floppy, problem
Topic #8: key, public, faq, mail, message, pgp, group, des, uni, sci
Topic #9: god, people, don't, jesus, believe, just, does, say, think, know



위 결과를 보고 토픽이 제대로 분류됐는지 확인하는 것을 분석가의 몫이다.

# 3. Gensim을 이용한 토픽 모델링

## 3.1 Gensim 사용법과 시각화

In [7]:
# pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 19.2 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


gensim은 텍스트에 대한 토큰화 결과를 입력으로 사용한다. 따라서 먼저 20 뉴스그룹의 문서들을 다음과 같이 토큰화한다.

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

cachedStopWords = stopwords.words('english')

RegTok = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english')) # 영어 불용어를 가져옴

def tokenizer(text):
    tokens = RegTok.tokenize(text.lower())
    # stopwords 제외
    words = [word for word in tokens if (word not in english_stops) and len(word) > 2]
    
    return words

texts = [tokenizer(news) for news in newsgroups_train.data]

* Gensim은 먼저 토큰화 결과로부터 토큰과 gensim 모듈이 내부적으로 사용하는 id를 매칭하는 사전을 생성한다. 이를 위한 클래스가 Dictionary이다.
* 다음 단계에서는 doc2bow() 메서드로 토큰화된 결과를 카운트 벡터, 즉 BOW 형태로 변환한다.

In [9]:
from gensim.corpora.dictionary import Dictionary

# 토큰화 결과로부터 dictionary 생성
dictionary = Dictionary(texts)
print('#Number of initial unique words in documents:', len(dictionary))

# 문서 빈도수가 너무 적거나 높은 단어를 필터링하고 특성을 단어의 빈도 순으로 선택
dictionary.filter_extremes(keep_n=2000, no_below=5, no_above=0.5)
print('#Number of unique words after removing rare and common words:', len(dictionary))

# 카운트 벡터로 변환
corpus = [dictionary.doc2bow(text) for text in texts]
print('#Number of unique tokens: %d' % len(dictionary))
print('#Number of documents: %d' % len(corpus))

#Number of initial unique words in documents: 46466
#Number of unique words after removing rare and common words: 2000
#Number of unique tokens: 2000
#Number of documents: 3219


In [10]:
from gensim.models import LdaModel

num_topics = 10
passes = 5
%time model = LdaModel(corpus=corpus, id2word=dictionary, \
                      passes=passes, num_topics=num_topics, \
                      random_state=7)

CPU times: user 15.9 s, sys: 0 ns, total: 15.9 s
Wall time: 16 s


상위 비중 단어 확인

In [11]:
model.print_topics(num_words=10)

[(0,
  '0.023*"com" + 0.018*"keith" + 0.016*"caltech" + 0.013*"sgi" + 0.013*"nntp" + 0.013*"posting" + 0.013*"host" + 0.012*"would" + 0.012*"system" + 0.011*"livesey"'),
 (1,
  '0.020*"morality" + 0.018*"objective" + 0.015*"one" + 0.015*"say" + 0.014*"uiuc" + 0.012*"frank" + 0.012*"values" + 0.010*"faq" + 0.010*"article" + 0.008*"cso"'),
 (2,
  '0.026*"com" + 0.025*"access" + 0.025*"posting" + 0.023*"host" + 0.023*"nntp" + 0.017*"digex" + 0.015*"article" + 0.013*"cwru" + 0.013*"___" + 0.013*"net"'),
 (3,
  '0.021*"university" + 0.017*"posting" + 0.015*"host" + 0.015*"nntp" + 0.013*"article" + 0.010*"com" + 0.009*"know" + 0.009*"i\'m" + 0.009*"would" + 0.009*"thanks"'),
 (4,
  '0.032*"com" + 0.015*"would" + 0.011*"article" + 0.010*"one" + 0.010*"get" + 0.009*"people" + 0.009*"ibm" + 0.008*"government" + 0.007*"good" + 0.007*"i\'m"'),
 (5,
  '0.025*"key" + 0.017*"encryption" + 0.014*"clipper" + 0.014*"chip" + 0.009*"keys" + 0.009*"use" + 0.008*"security" + 0.008*"government" + 0.008*"pub

In [13]:
import pyLDAvis

In [15]:
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# LDA 모형을 pyLDAvis 객체에 전달
lda_viz = gensimvis.prepare(model, corpus, dictionary)
lda_viz

/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.137200  0.040249       1        1  22.658984
5      0.065990  0.123619       2        1  14.525257
9      0.072133  0.092872       3        1  12.990073
7      0.138320  0.043608       4        1  11.020854
4     -0.021828  0.007494       5        1   9.293362
6      0.072783 -0.094709       6        1   9.122736
3      0.027375 -0.054949       7        1   8.495033
0     -0.040789 -0.129975       8        1   4.521932
2      0.005360 -0.091971       9        1   4.051670
1     -0.182145  0.063762      10        1   3.320099, topic_info=            Term         Freq        Total Category  logprob  loglift
538        space  1488.000000  1488.000000  Default  30.0000  30.0000
20           com  3751.000000  3751.000000  Default  29.0000  29.0000
834          key  1316.000000  1316.000000  Default  28.0000  28.0000
1551        scsi   741.000000   741.000000  Default  27.0000  27.0000
70       posting  1723.000000  1723.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
65        people    64.318179  1672.480228  Topic10  -5.1558   0.1470
789          sci    53.158361   309.408317  Topic10  -5.3464   1.6438
602   university    56.751576  1487.215137  Topic10  -5.2810   0.1392
294         time    55.658101  1187.675258  Topic10  -5.3004   0.3446
315       better    54.086295   564.454483  Topic10  -5.3291   1.0599

[592 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1359      1  0.006274      128
1359      2  0.069009      128
1359      3  0.037641      128
1359      4  0.878296      128
153       1  0.057459     1993
...     ...       ...      ...
343       8  0.022400    years
343       9  0.005600    years
343      10  0.121335    years
704       3  0.991291      zoo
705       3  0.983843  zoology

[2804 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 6, 10, 8, 5, 7, 4, 1, 3, 2])

/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/lib/python3.8/site-packages/sklearn/utils/multiclass.py:14: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1
/data/ydkim/.pyenv/versions/3.8.12/envs/py3.8.12/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `